## Data preparation

In [ ]:
import pandas as pd
from torchvision import transforms
import numpy as np
import random
import torch
import torch.nn as nn
import datetime
import os

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True  # Enforce deterministic algorithms
        torch.backends.cudnn.benchmark = False     # Disable benchmark for reproducibility

    os.environ['PYTHONHASHSEED'] = str(seed)       # Seed Python hashing, which can affect ordering
set_seed(42)

# DATASET_DIR = (Path("..") / ".." / "datasets").resolve()
# DATASETS = ["OFFICE-MANNERSDB", "MANNERSDBPlus"]
# LABEL_COLS = [
#     "Vaccum Cleaning", "Mopping the Floor", "Carry Warm Food",
#     "Carry Cold Food", "Carry Drinks", "Carry Small Objects",
#     "Carry Large Objects", "Cleaning", "Starting a conversation"
# ]

# import sys
# sys.path.append('..')
# from data_processing.data_processing import create_crossvalidation_loaders

import sys
sys.path.append('..')

from models.buffers import NaiveRehearsalBuffer
from data_processing.data_processing import get_domain_dataloaders
from models.training_utils import heuristic_dualbranch_batch, unified_train_loop
from models.heuristicSplitModel import DualBranchModel

## CL Models

In [ ]:
# Ideas for potential ways to combine networks
# residual = SpecificHead(base + invariant_feats)
# specific_feats = invariant_feats + residual

# gate = GateNet(base + invariant_feats)
# residual = SpecificHead(base)
# specific_feats = invariant_feats + gate * residual

# residual = Attention(invariant_feats, base)
# specific_feats = invariant_feats + residual

# gamma, beta = SpecificHead(base)
# specific_feats = gamma * invariant_feats + beta

# invariant_feats = InvariantHead(base)
# specific_feats = SpecificHead(base)
# cos_sim(invariant_feats, specific_feats) ~ 0


### K Fold Cross Validation

In [ ]:
# def combine_fold_histories(fold_history):
#     """
#     Combine multiple training histories by averaging them element-wise.
#     Handles both simple lists and lists of dictionaries.
#     """
#     combined_history = {}
#     metric_keys = list(fold_history[0].keys())
    
#     for key in metric_keys:
#         # Check if this metric contains dictionaries
#         first_element = fold_history[0][key][0] if fold_history[0][key] else None
        
#         if isinstance(first_element, dict):
#             # Handle lists of dictionaries (train_epoch_metrics, grad_norms)
#             combined_history[key] = average_list_of_dicts(fold_history, key)
#         else:
#             # Handle simple lists (train_epoch_loss, val_epoch_loss, cross_domain_val)
#             stacked_metrics = np.stack([fold_history[fold][key] for fold in fold_history])
#             combined_history[key] = np.mean(stacked_metrics, axis=0).tolist()
    
#     return combined_history

# def average_list_of_dicts(fold_history, metric_key):
#     """
#     Average a list of dictionaries across folds.
#     """
#     # Get the dictionary keys from the first fold's first epoch
#     dict_keys = list(fold_history[0][metric_key][0].keys())
    
#     # Convert each fold's list of dicts to a 2D numpy array
#     fold_arrays = []
#     for fold in fold_history:
#         # Convert list of dicts to 2D array: [epochs, sub_metrics]
#         fold_array = np.array([[epoch_dict[k] for k in dict_keys] 
#                               for epoch_dict in fold_history[fold][metric_key]])
#         fold_arrays.append(fold_array)
    
#     # Stack all folds and average: [folds, epochs, sub_metrics] -> [epochs, sub_metrics]
#     stacked = np.stack(fold_arrays)
#     averaged = np.mean(stacked, axis=0)
    
#     # Convert back to list of dictionaries
#     result = []
#     for epoch_values in averaged:
#         epoch_dict = dict(zip(dict_keys, epoch_values))
#         result.append(epoch_dict)
    
#     return result

In [ ]:
# from sklearn.model_selection import StratifiedKFold

# # Add K fold labels to datapoints in dataframe, later used for creating K Fold Dataloaders

# df = pd.read_pickle("../data/pepper_data.pkl")
# df['image_path'] = '../' + df['image_path']

# # Initialize fold column
# df['fold'] = -1

# # Get unique image paths for splitting
# unique_images_df = df[['image_path', 'domain']].reset_index(drop=True)
# #exclude test subset test idx get from get_dataloader()
# unique_images_df = unique_images_df[~unique_images_df['image_path'].isin(test_split_idx)]

# # Create stratified 5-fold splits based on domain
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# # Assign fold numbers based on domain stratification
# for fold, (_, val_idx) in enumerate(skf.split(unique_images_df['image_path'], unique_images_df['domain'])):
#     val_image_paths = unique_images_df.iloc[val_idx]['image_path'].tolist()
#     df.loc[df['image_path'].isin(val_image_paths), 'fold'] = fold

# # Verify domain distribution across folds
# print("Domain distribution across folds:")
# print(df.groupby(['fold', 'domain']).size().unstack(fill_value=0))

In [ ]:
# fold_loaders = create_crossvalidation_loaders(df, 5, batch_sizes=(32, 64, 64), resize_img_to=(512, 288))

## Training

### current

In [ ]:
# TODO tests: retrain the model with
# 3 trains averaged
# both branches + random masks

# but first: fix underfitting: validation set
# lower droput
# diff branch:
#     resnet18
#     mobilenetv2
#     efficientnetb0
#     clip

#     + ablations - only_soc, only_env, no_mask
# learning rate scheduler with warmup and decay

# then
# best model, x3 random initialisaiton, on same test set, no validation, report final metrics on test set
#same for ablations
#same for random mask

"""
mobilenetv2 is fine

test mobilenetv2 vs simple CNN
vs normalised features
vs 3 epochs

"""

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
        transforms.Resize((144,256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225])
    ])

domains = ['Home', 'BigOffice-2', 'BigOffice-3', 'Hallway', 'MeetingRoom', 'SmallOffice']

testing_scenarios = {
    'simplebranch':      (False, 30, {'branch':'simple'}),
    'simplebranch_branchnorm':      (True, 30, {'branch':'simple'}),
    'mobilenetv2_branchnorm':      (True, 30, {'branch':'mobilenetv2'}),
    'mobilenetv2':       (False, 3, {'branch':'mobilenetv2'}),
}
    
for name, (branch_norm, epochs, setup) in testing_scenarios.items():

        # Training Data
        df = pd.read_pickle("../data/pepper_data_train.pkl")

        domain_dataloaders = get_domain_dataloaders(df, batch_sizes=(32, 64, 64), double_img=True, transforms=[transform]*2, num_workers=0, include_test=None)

        print(f"\nTesting: {name}")
        model = DualBranchModel(dropout_rate=0.1, setup=setup, branch_norm=branch_norm)
        dual_model = model.to(device)
        trainable_params = [p for p in dual_model.parameters() if p.requires_grad]
        optimizer = torch.optim.Adam(trainable_params, lr=1e-3)
        buffer = NaiveRehearsalBuffer(buffer_size=120)

        exp_name = f"{name}_epochs{epochs}_dropout{0.1}__{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        dualbranch_kwargs = {
                'mse_criterion': nn.MSELoss(),
                'ce_criterion': nn.CrossEntropyLoss(),
            }
        
        
        unified_train_loop(
            model=dual_model,
            domains=domains,
            domain_dataloaders=domain_dataloaders,
            buffer=buffer,
            optimizer=optimizer,
            device=device,
            batch_fn=heuristic_dualbranch_batch,
            batch_kwargs=dualbranch_kwargs,
            num_epochs=epochs,
            exp_name=exp_name,
            gradient_clipping=True,
            collect_tsne_data=False,
            checkpoint_dir="../checkpoints",
            validation_set='val',
        )

In [ ]:
# import subprocess

# model_names = [
#     'heuristic_small_env',
#     'heuristic_square_img',
#     'heuristic_eval_buffer'
# ]

# for model_name in model_names:
#     print(f"Running {model_name}")
#     process = subprocess.Popen(
#         ["python", "train_models.py", "--model_name", model_name, "--num_workers", "0"],
#         stdout=subprocess.PIPE,
#         stderr=subprocess.STDOUT,
#         text=True
#     )

#     for line in process.stdout:
#         print(line, end='')

#     process.wait()